## Teste dos embeddings do t2vec usando a função de rank em Python

In [1]:
import h5py
import numpy as np

from sklearn.metrics.pairwise import cosine_distances

In [2]:
!ls '../data/'

D_p-pts.pickle	exp1-trj.h5	exp1-trj.t   README.md	   train.trg  vocab.txt
D_q-pts.pickle	exp1-trj.label	exp2-trj.h5  saved_models  val.src
Dq-pts.pickle	exp1-trj.pts	porto.csv    train.src	   val.trg


In [3]:
vecfile = '../data/exp1-trj.h5'

In [4]:
# Carregando os vetores (embeddings do t2vec)
with h5py.File(vecfile, 'r') as f:
    vecs = np.array(f['layer3'])

In [5]:
vecs[0]

array([-0.68647516,  0.86736786, -0.99379593, -1.        ,  0.9991714 ,
        0.6531223 ,  0.98044574,  0.9939335 , -0.7251372 , -1.        ,
       -0.03488281, -1.        ,  0.47257993, -0.8890793 ,  0.9831597 ,
        0.79987305, -1.        ,  0.992221  , -0.03797043, -0.90551114,
        0.39030176, -0.924447  , -0.13001299,  0.60980725, -0.16162528,
       -0.7791138 , -0.27359173,  1.        , -0.21262921, -0.5503183 ,
        0.10815368,  0.966651  ,  0.98836386,  1.        , -0.01773597,
        0.40998647,  0.9999997 ,  0.9998418 ,  0.75152695, -0.9231957 ,
       -0.9999996 ,  0.45086482,  0.54504293, -1.        , -0.6131711 ,
        0.72926766,  0.9999319 , -0.9115782 ,  0.09714752,  1.        ,
        0.7436581 , -0.51482934, -0.27191746, -0.6620586 ,  0.99229515,
       -0.6926462 ,  1.        ,  0.9596888 , -0.98361933,  0.9995927 ,
        0.0512807 ,  0.31783974, -0.9508169 ,  0.9912928 , -1.        ,
        0.22825116,  0.43637615, -0.59943515,  0.99894285, -0.41

In [6]:
type(vecs)

numpy.ndarray

In [7]:
len(vecs)

101000

In [11]:
query = vecs[:1000]
dbsearch = vecs[1000:101000]

In [12]:
print(len(query))
print(len(dbsearch))

1000
100000


In [13]:
def rank(t_query, i, db_search):
    dists = cosine_distances(t_query.reshape(1, -1), db_search) # pega todas as distâncias de Dq[i] as trajs do db_search (D_qUD_p)
    dists = dists.flatten()
    order = dists.argsort() # pega a ordem
    ranks = order.argsort() # pega o rank

    return ranks[i] + 1 # retorna o ranking de ta' no dbsearch | soma +1 pq o argsort rankea a partir de 0 (zero)

In [14]:
def mr(ranks):
    return (sum(ranks)/len(ranks))

In [17]:
%%time
# t2vec
dbsizes = [20000, 40000, 60000, 80000, 100000]
for dbsize in dbsizes:
    ranks = []
    search = dbsearch[:dbsize]
    for i in range(len(query)):
        ranks.append(rank(query[i], i, search)) # rank das ta no dbsearch!
    print('Mean rank: {} with dbsearch size: {}'.format(mr(ranks), dbsize))

Mean rank: 2.446 with dbsearch size: 20000
Mean rank: 3.692 with dbsearch size: 40000
Mean rank: 5.103 with dbsearch size: 60000
Mean rank: 6.878 with dbsearch size: 80000
Mean rank: 8.34 with dbsearch size: 100000
CPU times: user 15min 16s, sys: 38min 4s, total: 53min 21s
Wall time: 13min 26s
